# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="85%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent Agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.40.0 requires botocore==1.38.1, but you have botocore 1.39.11 which is incompatible.
awscli 1.40.0 requires s3transfer<0.13.0,>=0.12.0, but you have s3transfer 0.13.1 which is incompatible.
sagemaker-core 1.0.29 requires rich<14.0.0,>=13.0.0, but you have rich 14.0.0 which is incompatible.


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transposts](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are two standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particulary useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use custom transports implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll search for Amazon Bedrock pricing information to give you a concise overview.
Tool #1: search_documentation

Tool #2: read_documentation
Based on the AWS documentation, Amazon Bedrock uses a **pay-per-use pricing model** with the following key components:

**Main Pricing Structure:**
- **Token-based pricing** - You pay for input tokens (what you send) and output tokens (what the model generates)
- **Model-specific rates** - Different foundation models have different pricing tiers
- **No upfront costs** - You only pay for what you use

**Pricing Options:**
1. **On-Demand** - Pay per token with no commitments
2. **Provisioned Throughput** - Purchase dedicated capacity for consistent performance (higher cost but guaranteed availability)

**What You Pay For:**
- Foundation model inference requests
- Volume of tokens processed (both input and output)
- Provisioned Throughput units (if purchased)

The exact pricing varies by model provider (Anthropic, Meta, Cohere, etc.) and model size.

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First lets start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="85%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [6]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [31696]
INFO:     Waiting for application startup.


[05/20/25 10:39:33] INFO     StreamableHTTP session manager started                  ]8;id=743211;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=889560;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#109\109]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52768 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52768 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52784 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52790 - "GET /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52784 - "POST /mcp/ HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:52790 - "GET /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52798 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52798 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:60700 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60700 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:60704 - "DELETE /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:60704 - "DELETE /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:43938 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:43938 - "POST /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:43952 - "POST /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0

#### Integrating Streamable HTTP client with Agent

Now lets use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(tools=tools)

    response = str(agent("What is 2 + 2?"))

[05/20/25 10:39:45] INFO     Connecting to StreamableHTTP endpoint:                          ]8;id=631741;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=325537;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#461\461]8;;\
                             http://localhost:8000/mcp                                                             

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 307 Temporary   ]8;id=820912;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=432872;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Created new transport with session ID:                  ]8;id=75483;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=198455;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#229\229]8;;\
                             4a894f136f7b4b71b5aef24fe3116777                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=314194;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=364512;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 4a894f136f7b4b71b5aef24fe3116777           ]8;id=67873;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=808908;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#137\137]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=168356;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=199340;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\



The answer to 2 + 2 is 4.

[05/20/25 10:39:49] INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 307 Temporary ]8;id=326849;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=500322;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Terminating session: 4a894f136f7b4b71b5aef24fe3116777           ]8;id=606041;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=650503;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#661\661]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"      ]8;id=618903;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=637012;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool innvocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[05/20/25 10:40:01] INFO     Connecting to StreamableHTTP endpoint:                          ]8;id=699025;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=806289;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#461\461]8;;\
                             http://localhost:8000/mcp                                                             

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 307 Temporary   ]8;id=636915;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=74722;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Created new transport with session ID:                  ]8;id=715889;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=664646;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#229\229]8;;\
                             897e4c9dc1cb465686b9600c88cc67f5                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=512352;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=455218;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 897e4c9dc1cb465686b9600c88cc67f5           ]8;id=766737;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=825541;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#137\137]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=381044;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=135809;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Calculation result: 30


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 307 Temporary ]8;id=505693;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=653831;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Terminating session: 897e4c9dc1cb465686b9600c88cc67f5           ]8;id=186315;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=646143;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#661\661]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"      ]8;id=606075;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=1408;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

You can optinally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[05/20/25 10:42:20] INFO     Connecting to StreamableHTTP endpoint:                          ]8;id=714060;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=972160;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#461\461]8;;\
                             http://localhost:8000/mcp                                                             

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 307 Temporary   ]8;id=825641;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=619346;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Created new transport with session ID:                  ]8;id=955531;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=546259;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#229\229]8;;\
                             803b6de1480d455faa04a3c7a2902134                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=460032;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=457623;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 803b6de1480d455faa04a3c7a2902134           ]8;id=152298;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=676062;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#137\137]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp/ "HTTP/1.1 200 OK"        ]8;id=997663;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=587218;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Tool execution succeeded: Hello Amazon


[05/20/25 10:42:45] INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 307 Temporary ]8;id=872105;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=197947;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             Redirect"                                                                             

                    INFO     Terminating session: 803b6de1480d455faa04a3c7a2902134           ]8;id=820155;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=936561;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#661\661]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp/ "HTTP/1.1 200 OK"      ]8;id=173969;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=676815;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to show case this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [2]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

NameError: name 'MCPClient' is not defined

In [6]:
neptune_mcp_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.amazon-neptune-mcp-server@latest"], env = {"NEPTUNE_ENDPOINT":"neptune-db://db-neptune-1.cluster-cvczj9jgzh8p.us-west-2.neptune.amazonaws.com"}
        )
    )
)

In [14]:
cost_calculator_mcp_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.cost-analysis-mcp-server@latest"], env = {"AWS_PROFILE":"default"}
        )
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [8]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client, neptune_mcp_client, cost_calculator_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync() + neptune_mcp_client.list_tools_sync() + cost_calculator_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools, max_parallel_tools=1)

    response = agent(
        "Run a query in Neptune DB and summarize the purchase details in UK"
    )

I'll help you run a query to retrieve purchase details in the UK from Neptune DB. First, let's check the graph status and schema to understand what data we have available.
Tool #5: get_graph_status


Now, let's examine the graph schema to understand the structure and what vertices and edges are available:
Tool #6: get_graph_schema


Based on the schema, I can see that we have "Customer Purchase" nodes with a "Country_of_Residence" property, and they are connected to "Product Purchased" nodes via a "Purchased" relationship. Let's run an OpenCypher query to get purchase details for customers in the UK:
Tool #7: run_opencypher_query


Now, let's run another query that will help us create a more comprehensive summary of purchase patterns in the UK by aggregating the data:
Tool #8: run_opencypher_query


Let's also look at the most popular products in the UK:
Tool #9: run_opencypher_query


Let's also examine purchase patterns by time for UK customers:
Tool #10: run_opencypher_query




Bas

In [1]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client, neptune_mcp_client, cost_calculator_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync() + neptune_mcp_client.list_tools_sync() + cost_calculator_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools, max_parallel_tools=1)

    response = agent(
        "What is the cost for running Anthropic claude 3.7 model per month on a average with 100k tokens/day"
    )

NameError: name 'aws_docs_mcp_client' is not defined

In [19]:
# Connect to external MCP servers
quiz_service = MCPClient(lambda: streamablehttp_client("http://quiz-provider.com/mcp"))
library_service = MCPClient(lambda: streamablehttp_client("http://cs-library.edu/mcp"))

with quiz_service, library_server:
    # Combine all tools - they all work the same way!
    tools = (
        quiz_service.list_tools_sync() +      # Tools from the external quiz server
        library_service.list_tools_sync()  # Tools from the external library server
                           # Your custom tools
    )
    
    # Create agent with all tools
    agent = Agent(tools=tools)
    response = agent(
        "What are the list of tools available?"
    )

client failed to initialize
  + Exception Group Traceback (most recent call last):
  |   File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 101, in start
  |     self._init_future.result(timeout=30)
  |   File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/concurrent/futures/_base.py", line 458, in result
  |     return self.__get_result()
  |   File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
  |     raise self._exception
  |   File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 217, in _async_background_thread
  |     async with self._transport_callable() as (read_stream, write_stream, *_):
  |   File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/home/ec2-user/anaconda3/envs/python3

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent